# Transformer
We will implement the Transformer architecture presented in class.

In [67]:
import torch
import torch.nn as nn

We start with the attention. Define a class `TransformerAttention` that will contain all the functions related to the Transformer's attention that we need. Add an `__init__` method that takes `hidden_dim` and `num_heads` as parameters.

In [68]:
class TransformerAttention(nn.Module):
    
    def __init__(self, hidden_dim, num_heads):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        

Now we're adding its functions one after the other. We start with the best part: the attention function. Implement scaled-dot product attention when given `query`, `key`, and `value` tensors as inputs. The dimensions of these tensors are: `[batch_size, sequence_length, head_dim]`. Scaled dot-product attention is defined as:
$$\text{DPA}(Q, K, V) = \text{softmax}(\frac{Q K^\top}{\sqrt{d}}) V$$

In [69]:
import math
import torch.nn.functional as F

def dot_product_attention(self, query, key, value):
    return F.softmax((query @ key.transpose(-1, -2)) / math.sqrt(query.size(-1)), dim=-1) @ value

TransformerAttention.dot_product_attention = dot_product_attention

Implement a function `split_to_heads` that takes a tensor of dimensions `[?, ?, hidden_dim]` and splits it into `num_heads` tensors of size `[?, ?, head_dim]`, where $\text{head\_dim} = \frac{\text{hidden\_dim}}{\text{num\_heads}}$. The `?` dimensions are the same as before, but your implementation should be independent of them.

In [70]:
def split_to_heads(self, tensor: torch.Tensor) -> torch.Tensor:
    return tensor.split(self.hidden_dim // self.num_heads, dim=-1)

TransformerAttention.split_to_heads = split_to_heads

Now implement the `forward` method of `TransformerAttention` (and extend the `__init__` method if necessary). It should:
1. project its inputs into `query`, `key` and `value` tensors with 3 separate linear layers
2. split the tensors into chunks for each head to process
3. perform attention for each head separately
4. concatenate the results
5. run the output through another linear layer

Step 1 and 2 look reversed from the diagram we saw in class, but this is more intuitive and also how Hugging Face implements these operations.

In [71]:
def __init__(self, hidden_dim, num_heads):
    super(TransformerAttention, self).__init__()
    self.hidden_dim = hidden_dim
    self.num_heads = num_heads
    self.Q = nn.Linear(hidden_dim, hidden_dim)
    self.K = nn.Linear(hidden_dim, hidden_dim)
    self.V = nn.Linear(hidden_dim, hidden_dim)
    self.output_projection = nn.Linear(hidden_dim, hidden_dim)

def forward(self, x):
    q = self.Q(x)
    k = self.K(x)
    v = self.V(x)
    q = self.split_to_heads(q)
    k = self.split_to_heads(k)
    v = self.split_to_heads(v)
    attention = [self.dot_product_attention(q_, k_, v_) for q_, k_, v_ in zip(q, k, v)]
    attention = torch.cat(attention, dim=-1)
    return self.output_projection(attention)

TransformerAttention.__init__ = __init__
TransformerAttention.forward = forward

Create a class `TransformerAttentionBlock` that runs Transformer attention, then adds the input as a residual to the output and performs layer normalization.

In [72]:
class TransformerAttentionBlock(nn.Module):
    
    def __init__(self, hidden_dim, num_heads):
        super().__init__()
        self.attention = TransformerAttention(hidden_dim, num_heads)
        self.norm = nn.LayerNorm(hidden_dim)
    
    def forward(self, x):
        attention = self.attention(x)
        return self.norm(x + attention)

Create a class `FeedForwardNetwork` that consists of two linear layers with a ReLU in between. Also add a residual connection from the input to the output and apply layer normalization.

In [73]:
class FeedForwardNetwork(nn.Module):
    
    def __init__(self, hidden_dim, inner_dim):
        super().__init__()
        self.fc1 = nn.Linear(hidden_dim, inner_dim)
        self.fc2 = nn.Linear(inner_dim, hidden_dim)
        self.norm = nn.LayerNorm(hidden_dim)
    
    def forward(self, x):
        proj = F.relu(self.fc1(x))
        proj = self.fc2(proj)
        return self.norm(proj + x)

Now we can combine the `TransformerAttentionBlock` and the `FeedForwardNetwork` into a `TransformerLayer`. 

In [74]:
class TransformerLayer(nn.Module):
    
    def __init__(self, hidden_dim, ffn_inner_dim, num_heads):
        super().__init__()
        self.attention_block = TransformerAttentionBlock(hidden_dim, num_heads)
        self.feed_forward = FeedForwardNetwork(hidden_dim, ffn_inner_dim)
    
    def forward(self, x):
        x = self.attention_block(x)
        x = self.feed_forward(x)
        return x

We are ready to compose our `TransformerEncoder` of a given number of `TransformerLayer`s.

In [75]:
class TransformerEncoder(nn.Module):
    
    def __init__(self, hidden_dim, ffn_inner_dim, num_layers, num_heads):
        super().__init__()
        self.transformers = nn.ModuleList([TransformerLayer(hidden_dim, ffn_inner_dim, num_heads) for _ in range(num_layers)])
    
    def forward(self, x):
        for t in self.transformers:
            x = t(x)
        return x

Let's test our implementation with the hyperparameters...

In [76]:
hidden_dim = 20
embedding_dim = hidden_dim
ffn_dim = 100
num_heads = 4
num_encoder_layers = 6
batch_size = 2
x_len = 10

... and check if it produces the correct output shapes.

In [77]:
x = torch.randn(batch_size, x_len, embedding_dim)
encoder = TransformerEncoder(hidden_dim, ffn_dim, num_encoder_layers, num_heads)
output = encoder(x)
assert list(output.shape) == [batch_size, x_len, hidden_dim], "Wrong output shape"

## Transformer Decoder
For the Transformer decoder, two components are missing.
1. A causal mask in the `TransformerAttention`.
2. A cross-attention module in the `TransformerLayer`.

We start by generalizing the `TransformerAttention` class to use a causal mask in `dot_product_attention` if it is used for decoder self-attention. We check this by accessing an `is_decoder_self_attention` attribute of `self`, which we have to add as an argument to `TransformerAttention`'s `__init__` method first.

In [87]:
# Add an `is_decoder_self_attention` attribute to TransformerAttention.__init__
def __init__(self, hidden_dim, num_heads, is_decoder_self_attention=False):
    super(TransformerAttention, self).__init__()  # we get an error here if we call super().__init__()
    self.is_self_attention = is_decoder_self_attention
    self.hidden_dim = hidden_dim
    self.num_heads = num_heads
    self.Q = nn.Linear(hidden_dim, hidden_dim)
    self.K = nn.Linear(hidden_dim, hidden_dim)
    self.V = nn.Linear(hidden_dim, hidden_dim)
    self.output_projection = nn.Linear(hidden_dim, hidden_dim)

TransformerAttention.__init__ = __init__

In [104]:
# Change the dot_product attention to use a causal mask in case it is used in the decoder self-attention.
def dot_product_attention(self, query, key, value):
    attention_scores = (query @ key.transpose(-1, -2)) / math.sqrt(query.size(-1))
    if self.is_self_attention:
        mask = (1 - torch.ones_like(attention_scores).tril()).type(torch.bool)
        attention_scores.masked_fill_(mask, -1e8)
    return F.softmax(attention_scores, dim=-1) @ value

TransformerAttention.dot_product_attention = dot_product_attention

Now we add cross-attention. We do this by updating the `TransformerAttention`'s `forward` method to take `encoder_hidden_states` as an optional input. Check the lecture slides to see which input gets projected into queries, keys and values.

In [105]:
def forward(self, x, encoder_hidden_states=None):
    encoder_hidden_states = x if encoder_hidden_states is None else encoder_hidden_states
    q = self.Q(x)
    k = self.K(encoder_hidden_states)
    v = self.V(encoder_hidden_states)
    q = self.split_to_heads(q)
    k = self.split_to_heads(k)
    v = self.split_to_heads(v)
    attention = [self.dot_product_attention(q_, k_, v_) for q_, k_, v_ in zip(q, k, v)]
    attention = torch.cat(attention, dim=-1)
    return self.output_projection(attention)
    
TransformerAttention.forward = forward

We have to extend the `TransformerAttentionBlock` to allow that additional argument in its `forward` method.

In [106]:
def forward(self, x, encoder_hidden_states=None):
    attention = self.attention(x, encoder_hidden_states)
    return self.norm(x + attention)

TransformerAttentionBlock.forward = forward

Now we implement a `TransformerDecoderLayer` that consists of decoder self-attention, cross-attention and a feed-forward network. In the `forward` method, use the encoder hidden states as inputs to the cross-attention module.

In [107]:
class TransformerDecoderLayer(nn.Module):
    
    def __init__(self, hidden_dim, ffn_inner_dim, num_heads):
        super().__init__()
        self.self_attention = TransformerAttentionBlock(hidden_dim, num_heads)
        self.self_attention.attention.is_self_attention = True
        self.cross_attention = TransformerAttentionBlock(hidden_dim, num_heads)
        self.feed_forward = FeedForwardNetwork(hidden_dim, ffn_inner_dim)
    
    def forward(self, x, encoder_hidden_states):
        out = self.self_attention(x)
        out = self.cross_attention(out, encoder_hidden_states)
        return self.feed_forward(out)

Add a `TransformerDecoder` that holds the decoder layers.

In [108]:
class TransformerDecoder(nn.Module):
    
    def __init__(self, hidden_dim, ffn_inner_dim, num_layers, num_heads):
        super().__init__()
        self.layers = nn.ModuleList([TransformerDecoderLayer(hidden_dim, ffn_inner_dim, num_heads) for _ in range(num_layers)])
    
    def forward(self, x, encoder_hidden_states):
        for layer in self.layers:
            x = layer(x, encoder_hidden_states)
        return x

## Transformer Seq2seq Model
We can now put everything together. Create and instantiate a Transformer model that encodes a random input `x`, then generates an output hidden representation for each decoder input `y` that we could then feed into a classifier to predict the words.

In [109]:
class TransformerModel(nn.Module):
    
    def __init__(self, hidden_dim, ffn_dim, num_encoder_layers, num_decoder_layers, num_heads):
        super().__init__()
        self.encoder = TransformerEncoder(hidden_dim, ffn_dim, num_encoder_layers, num_heads)
        self.decoder = TransformerDecoder(hidden_dim, ffn_dim, num_decoder_layers, num_heads)
    
    def forward(self, x, y):
        encoder_hidden_states = self.encoder(x)
        return self.decoder(y, encoder_hidden_states)

We will use the following hyperparameters.

In [110]:
hidden_dim = 20
embedding_dim = hidden_dim
ffn_dim = 100
num_heads = 4
num_encoder_layers = 6
num_decoder_layers = 2
batch_size = 2
x_len = 10
y_len = 7

Now we can run our model and test that the output dimensions are correct.

In [111]:
x = torch.randn(batch_size, x_len, embedding_dim)
y = torch.randn(batch_size, y_len, embedding_dim)
model = TransformerModel(hidden_dim, ffn_dim, num_encoder_layers, num_decoder_layers, num_heads)
output = model(x, y)
assert list(output.shape) == [batch_size, y_len, hidden_dim], "Wrong output shape"
num_model_params = sum(param.numel() for param in model.parameters())
assert num_model_params == 50480, f"Wrong number of parameters: {num_model_params}"

## What is missing for a real implementation?
Look at the [implementation of the Transformer layer for BERT by HuggingFace](https://github.com/huggingface/transformers/blob/v4.46.0/src/transformers/models/bert/modeling_bert.py#L223), from line 223 until 641.

**Question:** Name the things you see HuggingFace's implementation do that is still missing in your own implementation.

**Answer:** 

* Dropout at numerous places
* Attention mask -> mask out attention to the padding tokens
* 
